<a href="https://colab.research.google.com/github/apirakqqqqq/GE337_Programming/blob/main/Week_1/LAB_1_%E0%B8%AD%E0%B8%A0%E0%B8%B4%E0%B8%A3%E0%B8%B1%E0%B8%81%E0%B8%A9%E0%B9%8C_%E0%B8%9B%E0%B8%B1%E0%B8%8D%E0%B8%8D%E0%B8%B2%E0%B8%AA%E0%B8%B2%E0%B8%84%E0%B8%A3_6606614870.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

point = gpd.read_file("/content/บางซื่อ.geojson")
polygon = gpd.read_file("/content/สถานีรถไฟในเขตบางซื่อ.geojson")


In [ ]:
stations_utm = point.to_crs(epsg=32647)

stations_utm["geometry"] = stations_utm.buffer(500)

buffer_areas = stations_utm.to_crs(epsg=4326)

In [ ]:
import geopandas as gpd
import folium


districts = gpd.read_file("/content/บางซื่อ.geojson")
stations = gpd.read_file("/content/สถานีรถไฟในเขตบางซื่อ.geojson")

print("District Columns:", districts.columns)
print("Station Columns:", stations.columns)


stations = stations.to_crs(districts.crs)

join = gpd.sjoin(stations, districts, how="left", predicate="within")


district_field = "ADM2_TH"

count_station = (
    join.groupby(district_field)
    .size()
    .reset_index(name="station_count")
)

districts = districts.merge(count_station, on=district_field, how="left")
districts["station_count"] = districts["station_count"].fillna(0)

print(districts[[district_field, "station_count"]])


stations_utm = stations.to_crs(epsg=32647)
stations_utm["geometry"] = stations_utm.buffer(500)
buffer_areas = stations_utm.to_crs(epsg=4326)

districts_utm = districts.to_crs(epsg=32647)
buffer_utm = buffer_areas.to_crs(epsg=32647)

buffer_union = buffer_utm.union_all()

districts_utm["buffer_area"] = districts_utm.geometry.intersection(buffer_union).area
districts_utm["district_area"] = districts_utm.geometry.area

districts_utm["coverage_percent"] = (
    districts_utm["buffer_area"] /
    districts_utm["district_area"] * 100
)

print(districts_utm[[district_field, "coverage_percent"]])


center = stations.geometry.unary_union.centroid

m = folium.Map(location=[center.y, center.x], zoom_start=13)

# polygon
folium.GeoJson(
    districts,
    name="District",
    style_function=lambda x: {
        "fillColor": "blue",
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.4,
    },
).add_to(m)

# จุดสถานี
folium.GeoJson(
    stations,
    name="Stations",
    marker=folium.CircleMarker(radius=5),
).add_to(m)

# buffer
folium.GeoJson(
    buffer_areas,
    name="Buffer 500m",
    style_function=lambda x: {
        "fillColor": "red",
        "color": "red",
        "weight": 1,
        "fillOpacity": 0.3,
    },
).add_to(m)

folium.LayerControl().add_to(m)


District Columns: Index(['FID', 'Shape_Leng', 'Shape_Area', 'ADM2_EN', 'ADM2_TH', 'ADM2_PCODE',
       'ADM1_EN', 'ADM1_TH', 'ADM1_PCODE', 'ADM0_EN', 'ADM0_TH', 'ADM0_PCODE',
       'geometry'],
      dtype='object')
Station Columns: Index(['FID', '_id', 'No_', 'Line_Code', 'Status', 'Station_Co', 'Station_Na',
       'Station__1', 'Latitude', 'Longitude', 'เขต', 'geometry'],
      dtype='object')
   ADM2_TH  station_count
0  บางซื่อ              6
   ADM2_TH  coverage_percent
0  บางซื่อ         28.833369


/tmp/ipython-input-3123431932.py:51: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = stations.geometry.unary_union.centroid


In [ ]:
m.save("bangsue_service_area_map.html")

ลิ้งนี้เอาไว้สำหรับดูภาพนะครับ https://apirakqqqqq.github.io/Service-Area-Analysis-of-Public-Transportation-Stations-in-Bang-Sue-District-in-Google-Colab/bangsue_service_area_map.html


**คำถามท้าย Lab**

**1. อธิบายความแตกต่างระหว่าง Spatial Join และ Attribute Join**

**ตอบ**
- Spatial Join คือ การเชื่อมข้อมูลโดยอาศัยความสัมพันธ์เชิงพื้นที่ (Spatial Relationship) เช่น จุดอยู่ภายในพื้นที่ใด (within), ซ้อนทับกัน (intersect) หรืออยู่ใกล้กัน (near) โดยใช้ geometry เป็นตัวกำหนดความสัมพันธ์
- Attribute Join คือ การเชื่อมข้อมูลโดยอาศัยค่าฟิลด์ที่เหมือนกันในตาราง เช่น รหัสเขตหรือชื่อเขตที่ตรงกัน โดยไม่ได้พิจารณาตำแหน่งทางภูมิศาสตร์

**2. ทำไมเราต้องแปลงระบบพิกัดก่อนใช้ buffer() ใน GeoPandas?**

**ตอบ**

คำสั่ง buffer() ใช้หน่วยตามระบบพิกัด (CRS) ของข้อมูล
ถ้าข้อมูลอยู่ในระบบพิกัดแบบภูมิศาสตร์ เช่น EPSG:4326 (Latitude/Longitude) หน่วยจะเป็น “องศา” ไม่ใช่ “เมตร” ทำให้ขนาดของ Buffer ไม่ถูกต้อง เช่น 500 จะหมายถึง 500 องศา ต้องแปลงระบบพิกัด เช่น UTM ก่อนจึงสร้าง Buffer ตามระยะที่ต้องการ

**3. หากต้องการวิเคราะห์พื้นที่ให้บริการของสถานีขนส่งในระยะ 10 กิโลเมตร แทน 5 กิโลเมตร ต้องแก้ไขส่วนใดของโค้ด?**

**ตอบ**

**stations_utm["geometry"] = stations_utm.buffer(500) = 500 M **
สามารถแก้ที่ตัวเลขได้เลยครับ เช่น อยากแปลงค่าจาก 5000 เป็น 5 KM สามารถแปลงที่ตัวเลขเป็น 10000 M และกดรัน ขนาด Buffer ก็จะเปลี่ยนเป็น 10 KM ครับ

**4. วิธีใดที่เหมาะสมที่สุดในการแสดงผลข้อมูลเชิงพื้นที่ในรูปแบบ Interactive Map? เพราะเหตุใด?**

**ตอบ**

การใช้ Folium เป็นวิธีที่เหมาะสมที่สุด เพราะสามารถแสดงผลข้อมูลเชิงพื้นที่ในรูปแบบแผนที่ที่โต้ตอบได้ เข้าใจง่าย และเหมาะสำหรับการวิเคราะห์และการนำเสนอผลการศึกษาเชิงพื้นที่ในสภาพแวดล้อมของ Python และ Google Colab

**5. หากพบว่า Buffer ที่สร้างมีขนาดไม่ถูกต้อง อาจเกิดจากสาเหตุใดได้บ้าง? และสามารถแก้ไขได้อย่างไร?**

**ตอบ**

**สาเหตุที่เป็นไปได้**

- ใช้ระบบพิกัดแบบ Latitude/Longitude (องศา) โดยไม่ได้แปลงเป็นหน่วยเมตรก่อน

- เลือก EPSG ผิดโซน (เช่น ใช้ UTM คนละโซน)

- ใส่ค่าระยะทางผิดหน่วย เช่น ใส่ 10 แทน 10,000

- ข้อมูลมี CRS ไม่ตรงกันก่อนทำการวิเคราะห์

**วิธีแก้ไข:**

- ตรวจสอบ CRS ของข้อมูลก่อนทุกครั้ง

- แปลงเป็นระบบพิกัดที่มีหน่วยเป็นเมตร (เช่น UTM) ก่อนใช้ buffer()

- ตรวจสอบค่าระยะทางว่าอยู่ในหน่วยเมตร

- ตรวจสอบให้แน่ใจว่า dataset ทั้งหมดใช้ CRS เดียวกัน